In [3]:
import pefile
from fastapi import FastAPI
import numpy as np
import pandas as pd

## Utility functions for server


In [2]:
import joblib

In [ ]:
import itertools
from sklearn.pipeline import Pipeline


def createDataframeFromPEdump(pe):

	dosHeaders = ['e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr',
	       'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs',
	       'e_lfarlc', 'e_ovno', 'e_oemid', 'e_oeminfo', 'e_lfanew']
	fileHeaders=['Machine',
	       'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable',
	       'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics']
	optionalHeaders=['Magic',
	       'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
	       'SizeOfInitializedData', 'SizeOfUninitializedData','AddressOfEntryPoint', 'BaseOfCode',
	        'ImageBase', 'SectionAlignment','FileAlignment', 'MajorOperatingSystemVersion',
	       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
	       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'SizeOfHeaders',
	       'CheckSum', 'SizeOfImage', 'Subsystem', 'DllCharacteristics',
	       'SizeOfStackReserve', 'SizeOfStackCommit', 'SizeOfHeapReserve',
	       'SizeOfHeapCommit', 'LoaderFlags', 'NumberOfRvaAndSizes']
	imageDirectory = ['ImageDirectoryEntryExport', 'ImageDirectoryEntryImport',
	       'ImageDirectoryEntryResource', 'ImageDirectoryEntryException',
	       'ImageDirectoryEntrySecurity']

	dheaders = {}
	fheaders = {}
	oheaders={}
	imd1 = {}

	for x in dosHeaders:
		dheaders[x] = getattr(pe.DOS_HEADER,x)
	df = pd.DataFrame(dheaders,index=[0])

	for i in fileHeaders:
		fheaders[i] = getattr(pe.FILE_HEADER,i)
	df = pd.concat([df,(pd.DataFrame(fheaders,index=[0]))],axis=1)

	for y in optionalHeaders:
		oheaders[y] = getattr(pe.OPTIONAL_HEADER,y)
	df = pd.concat([df,(pd.DataFrame(oheaders,index=[0]))],axis=1)

	for q in pe.OPTIONAL_HEADER.DATA_DIRECTORY:
	    imd1[q.name] = q.VirtualAddress
	imd1 = dict(itertools.islice(imd1.items(),5))
	df = pd.concat([df,(pd.DataFrame(imd1,index=[0]))],axis=1)

	return df

def getPredictions(df):
    load_scaler = joblib.load(open(r'../components/SScaler.pkl','rb'))
    load_skpca = joblib.load(open(r'../components/PCA.pkl','rb'))
    load_model = joblib.load(open(r'../components/model.pkl','rb'))
    pipe = Pipeline([('scale', load_scaler),('pca', load_skpca),('clf', load_model)])
    df = np.array(df)
    df = df.reshape(1,-1)
    results = pipe.predict_proba(df)
    pred = pipe.predict(df)
    return (results[0],pred[0])

## Exe to png

In [ ]:
import math
import os
from PIL import Image
def getBinaryData(filename):
	"""
	Extract byte values from binary executable file and store them into list
	:param filename: executable file name
	:return: byte value list
	"""

	binary_values = []

	with open(filename, 'rb') as fileobject:

		# read file byte by byte
		data = fileobject.read(1)

		while data != b'':
			binary_values.append(ord(data))
			data = fileobject.read(1)
	return binary_values

def createGreyScaleImage(filename, width=None):
    global img_path 
    greyscale_data  = getBinaryData(filename)
    size            = get_size(len(greyscale_data), width)
    # try:
    image = Image.new('L',size)
    image.putdata(greyscale_data)
      # setup output filename
    dirname     = os.path.dirname(filename)
    name, _     = os.path.splitext(filename)
    name        = os.path.basename(name)
    imagename   = dirname + os.sep + name + '_'+ '.png'
    os.makedirs(os.path.dirname(imagename), exist_ok=True)
    img_path = imagename
    image.save(imagename)
    print('The file', imagename, 'saved.')
    # except Exception as err:
    # 	  print(err)
  


def get_size(data_length, width=None):

	if width is None: # with don't specified any with value

		size = data_length

		if (size < 10240):
			width = 32
		elif (10240 <= size <= 10240 * 3):
			width = 64
		elif (10240 * 3 <= size <= 10240 * 6):
			width = 128
		elif (10240 * 6 <= size <= 10240 * 10):
			width = 256
		elif (10240 * 10 <= size <= 10240 * 20):
			width = 384
		elif (10240 * 20 <= size <= 10240 * 50):
			width = 512
		elif (10240 * 50 <= size <= 10240 * 100):
			width = 768
		else:
			width = 1024

		height = int(size / width) + 1

	else:
		width  = int(math.sqrt(data_length)) + 1
		height = width

	return (width, height)

In [ ]:
import numpy as np
from io import BytesIO
from keras.applications import imagenet_utils
import tensorflow as tf 
import itertools
import pefile
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import uvicorn
from fastapi import FastAPI, File, UploadFile
from starlette.responses import RedirectResponse
import cv2
import tempfile
import uvicorn
from keras.preprocessing.image import img_to_array
from fastapi.templating import Jinja2Templates

## PreProcess

In [ ]:
from io import BytesIO
from PIL import Image
model = None
output_dict = {'Adialer.C': 0,
 'Agent.FYI': 1,
 'Allaple.A': 2,
 'Allaple.L': 3,
 'Alueron.gen!J': 4,
 'Autorun.K': 5,
 'C2LOP.P': 6,
 'C2LOP.gen!g': 7,
 'Dialplatform.B': 8,
 'Dontovo.A': 9,
 'Fakerean': 10,
 'Instantaccess': 11,
 'Lolyda.AA1': 12,
 'Lolyda.AA2': 13,
 'Lolyda.AA3': 14,
 'Lolyda.AT': 15,
 'Malex.gen!J': 16,
 'Obfuscator.AD': 17,
 'Rbot!gen': 18,
 'Skintrim.N': 19,
 'Swizzor.gen!E': 20,
 'Swizzor.gen!I': 21,
 'VB.AT': 22,
 'Wintrim.BX': 23,
 'Yuner.A': 24}

def load_model():
  model =  tf.keras.models.load_model("../components//malware_model.h5")
  print("Model loaded...")
  return model
def cnn_predict(imagepath):
    global model
    if model is None:
        model = load_model()
    image = cv2.imread(imagepath)
    image = cv2.resize(image, (64, 64))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    res = model.predict_classes(image)
    ax = model.predict(image)
    return get_key(res)

def get_key(val):
    for key, value in output_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

## Fast API

In [ ]:
#creating fastApi app 
app_desc = """<h2> Try uploading a Portable Executable(PE) file"""
app = FastAPI(description = app_desc)
templates = Jinja2Templates(directory="/content/drive/MyDrive/Final year/Arsha")
@app.get("/",include_in_schema=False)
async def index():
	return RedirectResponse(url="/docs")

@app.post("/predict")
def parse(file: UploadFile = File(...)):
    extension = os.path.splitext(file.filename)[1]
    _, path = tempfile.mkstemp(prefix='parser_', suffix=extension)

    with open(path, 'ab') as f:
        for chunk in iter(lambda: file.file.read(10000), b''):
            f.write(chunk)

    # extract content
    content = pefile.PE(path,fast_load=True)
    img = None
    dataframe = createDataframeFromPEdump(content)
    binary_preds = getPredictions(dataframe)
    if binary_preds[1] == 1:
      createGreyScaleImage(path)
      img = cnn_predict(img_path)
      return {'response':'Malicious file','predictions':binary_preds[0][1]*100,'malware family':img}
    else:
      return {'Response': 'Your file is same from malware.','Malicious percentage':binary_preds[0][1]*100,'malware family':''}
    # remove temp file
    os.close(_)
    os.remove(path)
    


## future scope
